##### Copyright 2025 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用Gemini进行笔记整理


In [1]:
%pip install -U -q "google-genai>=1.16.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.6/222.6 kB 17.7 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)
MODEL_ID="gemini-2.5-flash-preview-05-20" # @param ["gemini-2.5-flash-lite", "gemini-2.5-flash", "gemini-2.5-pro", "gemini-2.5-flash-preview-05-20", "gemini-2.5-pro-preview-06-05"] {"allow-input":true, isTemplate: true}
import json
from PIL import Image
from IPython.display import display, Markdown

上传txt文件，整理得到相应笔记。

In [ ]:
# 文件读取
filename = r"/content/5660.txt"
with open(filename, 'r', encoding='utf-8') as f:
    text_content = f.read()

# 构造 prompt
prompt = f"""
请你将R这里的知识点进行整理，要求进行结构化编辑，放入一个md文件，每个单词下面有

meanings、usage、related word、notes共四条属性，word用一级标题，meaning如果有多个意思就标明词性，用bullet point呈现；usage也用bullet point呈现，每个usage的格式是"例子"（本单词在这个例子中的含意），这个例子可以是一个具体的句子，也可以是一个固定搭配；related word是它的衍生词、近义词、反义词等等有提到的相关词，如果它的relatedword有进一步解释，那么在这个单词下面列出它的related word，然后这个relatedword就另外用二级标题写出word后，剩下的四条属性按照相同规则编写，如果只是简单提一下relatedword，就在原单词下面简单描述一下即可。所有的phrase搭配、例句都分行列入usage里面；notes用来呈现词根词缀、记忆方法。

如果有提到这个属性，就用bulletpoint列出这个属性+：+低一级bulletpoint，如果没有则不提，如果无法归类到前三条属性，则归类为notes。

{text_content}
"""
thinking_budget = 4096 # @param {type:"slider", min:0, max:24576, step:1}

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            thinking_budget=thinking_budget
        )
    )
)

# 将 response.text 保存为 result.txt 文件
with open("result.txt", "w", encoding="utf-8") as f:
    f.write(response.text)

print("保存完成：result.txt")